In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
  ChatCompletionToolParam({
    "type": "function",
    "function": {
      "name": "get_products",
      "description": "グループごとに統一した商品名「group_name」と、グループに属する商品名のリスト「products」を結合した結果を返す",
      "parameters": {
        "type": "object",
        "properties": {
          "product_data": {
            "type": "array",
            "items": {
              "product_name": {
                "type": "string",
                "description": "グループごとに統一した商品名",
              },
              "products": {
                "type": "string",
                "description": "グループに属する商品名をカンマ区切りで並べた文字列",
              },
            },
          }
        },
        "required": ["product_name", "products"],
      },
    },
  })
]

print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='商品名')

# 2. データをLLM用にテキスト形式に変換
product_data_text = df.astype(str)
prompt_text = f"""
次の商品名のリストから、似た名前の商品をグループ化し、グループごとに統一した商品名「product_name」を提案し、グループに属する商品名をカンマ区切りで並べた文字列「products」を作成し、「product_nameとproductsのリスト」で構成されるJSONデータ「product_data」を出力してください。
{product_data_text}
"""

# 3. OpenAI APIの呼び出し
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_text},
    ],
    tools=tools,
    tool_choice={
        "type":"function",
        "function":{"name":"get_products"}
    },
    response_format={"type":"json_object"}
)

# 4. 分析結果をJSONオブジェクトに変換
# ツール呼出情報を取得
tool = response.choices[0].message.tool_calls[0]
# JSONオブジェクトに変換
data = json.loads(tool.function.arguments)

# データフレームに変換
df_out = pd.DataFrame(data['product_data'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("商品名グループ化結果.xlsx", index=False)

print("Excelファイルに処理結果を保存しました。")

処理を開始します。
Excelファイルに処理結果を保存しました。
